Rain Prediction in Australia - v1.1

### Introduction:
#### Using a rain dataset in Australia, we build an efficient classification model that predicts the next day's rain.
####  DataSet: https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import pickle
%matplotlib inline